$\newcommand{\t}[1]{\mathrm{#1}}$
$\newcommand{\e}[1]{\mathrm{#1}}$

# Example Bayesian Monte Carlo
### Kevin Kuns (2018)
This notebook gives an example of doing a Bayesian Monte Carlo analysis of a decaying sinusoidal function with noise. This uses the python packages emcee and corner. Install them in the same anaconda environment as finesse, etc.
```
conda install -c conda-forge emcee corner
```

In [ ]:
import numpy as np
import emcee
from numpy.random import randn, rand
import matplotlib.pyplot as plt
#import matplotlib as mpl
import matplotlib.gridspec as gridspec
import corner

# to check code execution times
from timeit import default_timer as timer

In [ ]:
#plt.style.use("../ranaIFO.mplstyle")

## Set up the model

As a model consider the observed data
$$y(t) = A e^{-t/\tau} \sin (2\pi ft) + \xi(t)$$
where $\xi(t)$ is noise assumed to be normally distributed with zero mean and unit variance. We want to infer the amplitude $A$, frequency $f$, and decay time $\tau$ from this noisy data.

In [ ]:
def signalFunction(theta, tt):
    A, f, tau = theta
    return A * np.exp(-tt/tau) * np.sin(2*np.pi*f*tt)


def lnlike(theta, tt, data):
    model = signalFunction(theta, tt)
    return -0.5 * np.sum((data - model)**2)


def lnprior(theta):
    A, f, tau = theta
    if tau < 0.1 or tau > 10:
        return -np.inf
    if f < 0.1 or f > 10:
        return -np.inf
    if A < 0:
        return -np.inf
    else:
        return -1/(2*5) * (A - 10)**2


def lnpost(theta, tt, data):
    return lnlike(theta, tt, data) + lnprior(theta)

In [ ]:
# Define the signal and data
tt = np.linspace(0, 15, 500)
A = 7
f = 0.5
tau = 5
signal = signalFunction([A, f, tau], tt)
noise = randn(len(tt))
data = signal + noise

In [ ]:
fig,ax = plt.subplots()
ax.plot(tt, data, label='data')
ax.plot(tt, signal, label='signal')
ax.set_xlabel('Time [s]')
ax.set_ylabel('Amplitude [arb]')
ax.set_xlim(tt[0], tt[-1])
ax.legend()
ax.grid('on', which='both', alpha=0.3)
ax.set_title("Siddesh's Fav Function");

## Setup the Monte Carlo Sampling

In [ ]:
ndim = 3 # number of parameters to estimate
nwalkers = 900 # number of walkers
nsteps = 1000 # number of steps each walker will take
nburn = int(nsteps/4) # number of steps to "burn in"
nthreads = 3 # number of parallel threads to use

In [ ]:
# Choose initial conditions
theta0 = np.array(
    [[1 + 0.01*randn(), 1 + 0.01*randn(), 1 + rand()]
     for ii in range(nwalkers)])

In [ ]:
# Run the sampler
t_0 = timer()
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnpost, args=(tt, data), threads=nthreads)
sampler.run_mcmc(theta0, nsteps)
t_elapsed = timer() - t_0
print('Elapsed time = {t:4.1f} seconds.'.format(t=t_elapsed))

## Plot the Results

In [ ]:
fig = plt.figure(figsize=(11, 18))
gs = gridspec.GridSpec(3, 1, hspace=0.05)
A_ax = fig.add_subplot(gs[0])
f_ax = fig.add_subplot(gs[1], sharex=A_ax)
tau_ax = fig.add_subplot(gs[2], sharex=A_ax)
for ii in range(0, nwalkers, int(nwalkers/10)):
    A_ax.plot(sampler.chain[ii, :, 0])
    f_ax.plot(sampler.chain[ii, :, 1])
    tau_ax.plot(sampler.chain[ii, :, 2])
plt.setp(A_ax.get_xticklabels(), visible=False)
plt.setp(f_ax.get_xticklabels(), visible=False)
A_ax.set_ylabel(r'$Amplitude$')
f_ax.set_ylabel(r'$freq$')
tau_ax.set_ylabel(r'$\tau$')
tau_ax.set_xlabel('step')
tau_ax.set_xlim(0, nsteps)
for ax in [A_ax, f_ax, tau_ax]:
    ax.grid('on', which='both', alpha=0.3)
    ax.grid(which='minor', alpha=0.2)

In [ ]:
# Only take the samples after burning in the sampler to remove effects of initial conditions
samples = sampler.chain[:, nburn:, :].reshape((-1, ndim))

In [ ]:
# Make a corner plot
fig = corner.corner(samples, labels=[r'$A$', r'$f$', r'$\tau$'], truths=[A, f, tau])
fig.set_size_inches((12, 12))

In [ ]:
# This is a more complicated but prettier plot
ll = 0.75
ul = 1.25
TV = [A, f, tau]
fig,ax = plt.subplots(np.shape(samples)[1],
                      np.shape(samples)[1], figsize=(12,12))
corner.corner(samples,
        labels=['Amplitude [arb]',
            'Frequency [Hz]',
            'Decay Time Constant [s]'],
            #quantiles=[0.9, 0.95, 0.98],
            truths = TV,
            truth_color = 'xkcd:bright orange',
            show_titles = True,
            use_math_text = True,
            bins = 50,
            range = [(ll*A, ul*A), (ll*f, ul*f), (ll*tau, ul*tau)],
            #   levels=(0.95,),
            color = 'xkcd:irish green',
            smooth = 0.5, # smoothing scale in std's
            hist_kwargs  = {'linewidth':2.5},
            label_kwargs = {'fontsize':'large', 'fontweight':'bold'},
            title_kwargs = {'fontsize':'medium', 'fontweight':'bold'},
                  fig = fig)

for i in range(ndim):
    for j in range(ndim):
        ax[i,j].set_xlim(ll*TV[j], ul*TV[j])
        if i != j:
            ax[i,j].set_ylim(ll*TV[i], ul*TV[i])
        
    

# Print the MC parameters onto the plot
#ax[0,2].text(0.005,0.27, parText, wrap=True,transform=ax[0,2].transAxes)
pdfFile = 'MCMCposterior' + '.pdf'

try:   # try to print the file
    fig.savefig(pdfFile)
    #print("File saved as " + pdfFile)
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise
